<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://github.com/rsceth/myanmar_ai_platform"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

**Installation**

In [ ]:
! pip install transformers

In [ ]:
! pip install datasets

In [ ]:
! pip install evaluate

## Transfer Learning

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, BertModel
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import pipeline
from datasets import list_datasets, load_dataset
from sklearn.model_selection import train_test_split


import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

def preprocess_function(sentence):
    return tokenizer(sentence["text"], truncation=True)

In [ ]:
df = load_dataset('carblacac/twitter-sentiment-analysis')  

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
data = df["train"].train_test_split(test_size = 0.1)["test"]

In [ ]:
data = data.to_pandas()

In [ ]:
train, test = train_test_split(data, test_size = 0.2)

In [ ]:
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

((9599, 2), (2400, 2))

In [ ]:
modelType = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(modelType)
model = AutoModelForSequenceClassification.from_pretrained(
    modelType
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu"

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/pytorch_model.bin
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a Ber

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataset = train_dataset.rename_column("feeling", "labels")
train_dataset = train_dataset.with_format("torch")

test_dataset = test_dataset.rename_column("feeling", "labels")
test_dataset = test_dataset.with_format("torch")

Map:   0%|          | 0/9599 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="SentimentModel",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
train_dataset

Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9599
})

In [ ]:
trainer = Trainer(model=model,
    args=training_args,
    train_dataset=train_dataset,  # to check transfer learning ability , dataset is reduced
    eval_dataset=test_dataset, # to check transfer learning ability , dataset is reduced
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9599
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1200
  Number of trainable parameters = 108311810


Epoch,Training Loss,Validation Loss,Accuracy
1,0.395900,0.367831,0.823333
2,0.280500,0.456432,0.823333


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2400
  Batch size = 16
Saving model checkpoint to SentimentModel/checkpoint-600
Configuration saved in SentimentModel/checkpoint-600/config.json
Model weights saved in SentimentModel/checkpoint-600/pytorch_model.bin
tokenizer config file saved in SentimentModel/checkpoint-600/tokenizer_config.json
Special tokens file saved in SentimentModel/checkpoint-600/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely

TrainOutput(global_step=1200, training_loss=0.31930831909179686, metrics={'train_runtime': 256.8817, 'train_samples_per_second': 74.735, 'train_steps_per_second': 4.671, 'total_flos': 464863276499220.0, 'train_loss': 0.31930831909179686, 'epoch': 2.0})

### Zero Shot Learning

In [2]:
modelType = "xlm-roberta-large"

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
classifier = pipeline("zero-shot-classification", model =modelType)
candidate_labels = ["နည်းပညာ","ကျန်းမာရေး", "အားကစား", "စီးပွားရေး"]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [9]:
classifier("လန်ဒန် ကို ပြောင်း လာ ပြီး အာဆင်နယ် ကို ကိုယ်စားပြု ပရီးမီးယားလိဂ် မှာ ကစား ဖို့ ကျွန်တော် မ စောင့် နိုင် တော့ ပါ ဘူး ဟု ဆာ ခါ က အာဆင်နယ် ဝက်ဘ်ဆိုက် တွင် ရေးသား သည်", 
           candidate_labels)

{'sequence': 'လန်ဒန် ကို ပြောင်း လာ ပြီး အာဆင်နယ် ကို ကိုယ်စားပြု ပရီးမီးယားလိဂ် မှာ ကစား ဖို့ ကျွန်တော် မ စောင့် နိုင် တော့ ပါ ဘူး ဟု ဆာ ခါ က အာဆင်နယ် ဝက်ဘ်ဆိုက် တွင် ရေးသား သည်',
 'labels': ['အားကစား', 'ကျန်းမာရေး', 'နည်းပညာ', 'စီးပွားရေး'],
 'scores': [0.2502340078353882,
  0.25009217858314514,
  0.249860480427742,
  0.24981339275836945]}

In [10]:
classifier("ဝန်ထမ်း အပြောင်းအရွှေ့ ကိစ္စ နှင့် ပတ်သက် ပြီး မြန်မာ့ ရေနံ ထွက် လုပ်ငန်း နှင့် စင်ကာပူ အခြေစိုက် ပူး မား ကုမ္ပဏီ တို့ ပူးပေါင်း လုပ်ကိုင် နေ သော NEPAS ကုမ္ပဏီ မှ တာဝန်ရှိသူ များ နှင့် ဝန်ထမ်း များ ပူးပေါင်း ၍ မေလ ၂၀ ရက် တွင် မြန်မာ ကွန် ဗင်း ရှင်း စင်တာ တွင် သတင်းစာ ရှင်းလင်းပွဲ ကျင်းပသည်",
           candidate_labels)

{'sequence': 'ဝန်ထမ်း အပြောင်းအရွှေ့ ကိစ္စ နှင့် ပတ်သက် ပြီး မြန်မာ့ ရေနံ ထွက် လုပ်ငန်း နှင့် စင်ကာပူ အခြေစိုက် ပူး မား ကုမ္ပဏီ တို့ ပူးပေါင်း လုပ်ကိုင် နေ သော NEPAS ကုမ္ပဏီ မှ တာဝန်ရှိသူ များ နှင့် ဝန်ထမ်း များ ပူးပေါင်း ၍ မေလ ၂၀ ရက် တွင် မြန်မာ ကွန် ဗင်း ရှင်း စင်တာ တွင် သတင်းစာ ရှင်းလင်းပွဲ ကျင်းပသည်',
 'labels': ['စီးပွားရေး', 'အားကစား', 'ကျန်းမာရေး', 'နည်းပညာ'],
 'scores': [0.2509593367576599,
  0.2505514323711395,
  0.24997489154338837,
  0.2485143542289734]}

In [11]:
classifier("လန်းဆန်းကြည်လင်ရွှင်ပျစွာ အလုပ်လုပ်လိုလျှင် အိပ်ရေးဝအောင် အိပ်သင့်သည်။ လူ၏ကိုယ်ခန္ဓာ စက်ယန္တာရားကြီးကို အလုပ်ပြီးသည့်အခါ ရပ်နားအိပ်စက်ခွင့် ပေးရလေသည်",
           candidate_labels)

{'sequence': 'လန်းဆန်းကြည်လင်ရွှင်ပျစွာ အလုပ်လုပ်လိုလျှင် အိပ်ရေးဝအောင် အိပ်သင့်သည်။ လူ၏ကိုယ်ခန္ဓာ စက်ယန္တာရားကြီးကို အလုပ်ပြီးသည့်အခါ ရပ်နားအိပ်စက်ခွင့် ပေးရလေသည်',
 'labels': ['ကျန်းမာရေး', 'စီးပွားရေး', 'အားကစား', 'နည်းပညာ'],
 'scores': [0.25164201855659485,
  0.24954234063625336,
  0.2495308220386505,
  0.2492848038673401]}

In [12]:
classifier("ယခု ခေတ်အခါ အဆုတ်နှင့် ဆိုင်သော ရောဂါများကို ကုသရာ၌ ဆရာဝန်တို့သည် လေကောင်းလေသန့် ရရှိရန်ကို ဦးစွာ အရေးထားကြရာ အချို့ လူနာများကို အိမ်ပြင်၌ပင် အိပ်စက်ခိုင်းသည်",
           candidate_labels)

{'sequence': 'ယခု ခေတ်အခါ အဆုတ်နှင့် ဆိုင်သော ရောဂါများကို ကုသရာ၌ ဆရာဝန်တို့သည် လေကောင်းလေသန့် ရရှိရန်ကို ဦးစွာ အရေးထားကြရာ အချို့ လူနာများကို အိမ်ပြင်၌ပင် အိပ်စက်ခိုင်းသည်',
 'labels': ['ကျန်းမာရေး', 'အားကစား', 'နည်းပညာ', 'စီးပွားရေး'],
 'scores': [0.2537323832511902,
  0.24935029447078705,
  0.24863465130329132,
  0.24828267097473145]}

In [13]:
classifier("ရုရှားကို စစ်လက်နက် ခဲယမ်းတွေ ကူညီဖို့ ပြင်ဆင်နေတယ်ဆိုတဲ့ အမေရိကန်နိုင်ငံခြားရေးဝန်ကြီးရဲ့ ပြောဆိုချက်ကို လက်မခံနိုင်ကြောင်း တရုတ်နိုင်ငံခြားရေးဝန်ကြီးဌာန ပြောခွင့်ရသူက ဖေဖော်ဝါရီ ၂၀ ရက် သတင်းစာရှင်းလင်းပွဲမှာ ပြောလိုက်ပါတယ်,",
           candidate_labels)

{'sequence': 'ရုရှားကို စစ်လက်နက် ခဲယမ်းတွေ ကူညီဖို့ ပြင်ဆင်နေတယ်ဆိုတဲ့ အမေရိကန်နိုင်ငံခြားရေးဝန်ကြီးရဲ့ ပြောဆိုချက်ကို လက်မခံနိုင်ကြောင်း တရုတ်နိုင်ငံခြားရေးဝန်ကြီးဌာန ပြောခွင့်ရသူက ဖေဖော်ဝါရီ ၂၀ ရက် သတင်းစာရှင်းလင်းပွဲမှာ ပြောလိုက်ပါတယ်,',
 'labels': ['အားကစား', 'ကျန်းမာရေး', 'နည်းပညာ', 'စီးပွားရေး'],
 'scores': [0.25041595101356506,
  0.24996553361415863,
  0.2498147338628769,
  0.24980373680591583]}